# <center> <img src="../../labs/img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
### <center> **Procesamiento de Datos Masivos** </center>
---
### <center> **Primavera 2025** </center>
---
**Primer Examen**

**Fecha**: 14 de Marzo del 2025

**Nombre del estudiante**: Tirzah Peniche Barba

**Professor**: Pablo Camarillo Ramirez

In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("SparkSQL-Exam-1-Tirzah") \
    .master("spark://ac7f0d7e8e91:7077") \
    .config("spark.ui.port","4040") \
    .getOrCreate()
sc = spark.sparkContext

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/14 13:40:37 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
from equipo.spark_utils import SparkUtils

departments_schema = SparkUtils.generate_schema([("department_id", "integer"), ("department_name", "string"), ("location", "string")])

departments_df = spark.read \
                .schema(departments_schema) \
                .option("header", "true") \
                .csv("/home/jovyan/notebooks/data/exam_P2025_ESI3914B/departments.csv")

departments_df.printSchema()

departments_df.show(5, truncate=False)

root
 |-- department_id: integer (nullable = true)
 |-- department_name: string (nullable = true)
 |-- location: string (nullable = true)



+-------------+----------------------+---------+
|department_id|department_name       |location |
+-------------+----------------------+---------+
|101          |Human Resources       |San Diego|
|102          |Finance and Accounting|New York |
|103          |Sales and Marketing   |Chicago  |
|104          |Data Engineering      |Zapopan  |
|105          |Data Science          |Seattle  |
+-------------+----------------------+---------+
only showing top 5 rows



In [22]:
from equipo.spark_utils import SparkUtils

employees_schema = SparkUtils.generate_schema([("employee_id", "integer"), ("employee_info", "string")])

employees_df = spark.read \
                .schema(employees_schema) \
                .option("header", "true") \
                .csv("/home/jovyan/notebooks/data/exam_P2025_ESI3914B/employees.csv")

employees_df.printSchema()

employees_df.show(5, truncate=False)

root
 |-- employee_id: integer (nullable = true)
 |-- employee_info: string (nullable = true)

+-----------+-------------------------------------------------------------------------------------------+
|employee_id|employee_info                                                                              |
+-----------+-------------------------------------------------------------------------------------------+
|1          |{'name': 'Caitlyn', 'department_id': 103, 'salary': 115959.78, 'hire_date': '2002-06-10'}  |
|2          |{'name': 'Rachel', 'department_id': 104, 'salary': 100820.16, 'hire_date': '2009-07-01'}   |
|3          |{'name': 'Carrie', 'department_id': 105, 'salary': 114421.44, 'hire_date': '1998-12-10'}   |
|4          |{'name': 'Renee', 'department_id': 104, 'salary': 54688.13, 'hire_date': '1995-03-17'}     |
|5          |{'name': 'Gabriella', 'department_id': 109, 'salary': 106267.03, 'hire_date': '1995-02-09'}|
+-----------+--------------------------------------------

In [23]:
from pyspark.sql.functions import get_json_object


employees_df = employees_df.withColumn("name", get_json_object(employees_df.employee_info, "$.name").cast("string")) \
                            .withColumn("department_id", get_json_object(employees_df.employee_info, "$.department_id").cast("int")) \
                            .withColumn("salary", get_json_object(employees_df.employee_info, "$.salary").cast("double")) \
                            .withColumn("hire_date", get_json_object(employees_df.employee_info, "$.hire_date").cast("date"))

employees_df = employees_df.select("employee_id", "name", "department_id", "salary", "hire_date")

employees_df.printSchema()

employees_df.show(truncate=False)

root
 |-- employee_id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- department_id: integer (nullable = true)
 |-- salary: double (nullable = true)
 |-- hire_date: date (nullable = true)

+-----------+---------+-------------+---------+----------+
|employee_id|name     |department_id|salary   |hire_date |
+-----------+---------+-------------+---------+----------+
|1          |Caitlyn  |103          |115959.78|2002-06-10|
|2          |Rachel   |104          |100820.16|2009-07-01|
|3          |Carrie   |105          |114421.44|1998-12-10|
|4          |Renee    |104          |54688.13 |1995-03-17|
|5          |Gabriella|109          |106267.03|1995-02-09|
|6          |Kristen  |101          |88237.54 |2010-11-15|
|7          |Jonathan |102          |39323.42 |2012-06-30|
|8          |Michelle |101          |64262.85 |2005-10-30|
|9          |Michelle |105          |103521.88|1991-07-10|
|10         |Lisa     |110          |55435.93 |2016-03-25|
|11         |Cheryl   |1

In [24]:
employees_df = employees_df.join(departments_df, employees_df.department_id == departments_df.department_id, "inner")

employees_df.show()

+-----------+---------+-------------+---------+----------+-------------+--------------------+-------------+
|employee_id|     name|department_id|   salary| hire_date|department_id|     department_name|     location|
+-----------+---------+-------------+---------+----------+-------------+--------------------+-------------+
|          1|  Caitlyn|          103|115959.78|2002-06-10|          103| Sales and Marketing|      Chicago|
|          2|   Rachel|          104|100820.16|2009-07-01|          104|    Data Engineering|      Zapopan|
|          3|   Carrie|          105|114421.44|1998-12-10|          105|        Data Science|      Seattle|
|          4|    Renee|          104| 54688.13|1995-03-17|          104|    Data Engineering|      Zapopan|
|          5|Gabriella|          109|106267.03|1995-02-09|          109|    Customer Service|San Francisco|
|          6|  Kristen|          101| 88237.54|2010-11-15|          101|     Human Resources|    San Diego|
|          7| Jonathan|     

In [27]:
from pyspark.sql.functions import col, when

employees_df = employees_df.withColumn("salary_category", when(col("salary") >= 55000, "High").otherwise("Low"))

employees_df.show(5, truncate=False)

+-----------+---------+-------------+---------+----------+-------------+-------------------+-------------+---------------+
|employee_id|name     |department_id|salary   |hire_date |department_id|department_name    |location     |salary_category|
+-----------+---------+-------------+---------+----------+-------------+-------------------+-------------+---------------+
|1          |Caitlyn  |103          |115959.78|2002-06-10|103          |Sales and Marketing|Chicago      |High           |
|2          |Rachel   |104          |100820.16|2009-07-01|104          |Data Engineering   |Zapopan      |High           |
|3          |Carrie   |105          |114421.44|1998-12-10|105          |Data Science       |Seattle      |High           |
|4          |Renee    |104          |54688.13 |1995-03-17|104          |Data Engineering   |Zapopan      |Low            |
|5          |Gabriella|109          |106267.03|1995-02-09|109          |Customer Service   |San Francisco|High           |
+-----------+---

In [28]:
high_salary_df = employees_df.filter(col("salary_category") == "High")

low_salary_df = employees_df.filter(col("salary_category") == "Low")

high_salary_df.show(5, truncate=False)

low_salary_df.show(5, truncate=False)

+-----------+---------+-------------+---------+----------+-------------+-------------------+-------------+---------------+
|employee_id|name     |department_id|salary   |hire_date |department_id|department_name    |location     |salary_category|
+-----------+---------+-------------+---------+----------+-------------+-------------------+-------------+---------------+
|1          |Caitlyn  |103          |115959.78|2002-06-10|103          |Sales and Marketing|Chicago      |High           |
|2          |Rachel   |104          |100820.16|2009-07-01|104          |Data Engineering   |Zapopan      |High           |
|3          |Carrie   |105          |114421.44|1998-12-10|105          |Data Science       |Seattle      |High           |
|5          |Gabriella|109          |106267.03|1995-02-09|109          |Customer Service   |San Francisco|High           |
|6          |Kristen  |101          |88237.54 |2010-11-15|101          |Human Resources    |San Diego    |High           |
+-----------+---

In [ ]:
high_salary_df.createOrReplaceTempView("high_salary")

spark.sql("SELECT department_name, avg(salary) as avg_salary FROM high_salary GROUP BY department_name").show()


+--------------------+------------------+
|     department_name|        avg_salary|
+--------------------+------------------+
|Corporate Strateg...|102741.38324414717|
| Sales and Marketing|100839.65275449108|
|    Data Engineering|101626.29492163012|
|Research and Deve...|  98714.3003086419|
|Finance and Accou...|100731.07877887784|
|    Customer Service|101585.01600000002|
|               Legal|  99366.3129102167|
|        Data Science|101903.63710344829|
|          Operations|100169.65621722837|
|     Human Resources|104999.43191489363|
+--------------------+------------------+



In [35]:
low_salary_df.createOrReplaceTempView("low_salary")

spark.sql("SELECT department_name, avg(salary) as avg_salary FROM low_salary GROUP BY department_name").show()


+--------------------+------------------+
|     department_name|        avg_salary|
+--------------------+------------------+
|Corporate Strateg...|41590.741833333326|
| Sales and Marketing| 41150.40277777778|
|    Data Engineering| 41358.50794117647|
|Finance and Accou...|42740.952888888874|
|Research and Deve...| 41426.43521126761|
|    Customer Service|42644.472021276604|
|               Legal| 41160.26616438357|
|        Data Science| 41974.18958333334|
|          Operations|40646.100705882345|
|     Human Resources| 41751.64784810126|
+--------------------+------------------+



In [39]:
top_high_salary = spark.sql("SELECT name, salary FROM high_salary ORDER BY salary DESC LIMIT 5").show()

+---------+---------+
|     name|   salary|
+---------+---------+
|Gabriella|149989.73|
|Katherine| 149979.3|
|     Ryan| 149963.1|
|  Caitlyn|149956.54|
|     Mark|149915.56|
+---------+---------+



In [38]:
top_low_salary = spark.sql("SELECT name, salary FROM low_salary ORDER BY salary DESC LIMIT 5").show()

+-----+--------+
| name|  salary|
+-----+--------+
|Linda|54993.53|
|Tammy|54991.71|
|Aaron|54989.45|
|Craig| 54945.2|
|Aaron| 54937.3|
+-----+--------+



In [40]:
from pyspark.sql.functions import current_date, datediff

employees_df = employees_df.withColumn("years_in_company", datediff(current_date(), col("hire_date")) / 365)

employees_df.show(5, truncate=False)

+-----------+---------+-------------+---------+----------+-------------+-------------------+-------------+---------------+------------------+
|employee_id|name     |department_id|salary   |hire_date |department_id|department_name    |location     |salary_category|years_in_company  |
+-----------+---------+-------------+---------+----------+-------------+-------------------+-------------+---------------+------------------+
|1          |Caitlyn  |103          |115959.78|2002-06-10|103          |Sales and Marketing|Chicago      |High           |22.775342465753425|
|2          |Rachel   |104          |100820.16|2009-07-01|104          |Data Engineering   |Zapopan      |High           |15.712328767123287|
|3          |Carrie   |105          |114421.44|1998-12-10|105          |Data Science       |Seattle      |High           |26.276712328767122|
|4          |Renee    |104          |54688.13 |1995-03-17|104          |Data Engineering   |Zapopan      |Low            |30.013698630136986|
|5    

In [44]:
employees_older_df = employees_df.filter(col("years_in_company") > 15)
employees_older_df.show(5, truncate=False)


+-----------+---------+-------------+---------+----------+-------------+-------------------+-------------+---------------+------------------+
|employee_id|name     |department_id|salary   |hire_date |department_id|department_name    |location     |salary_category|years_in_company  |
+-----------+---------+-------------+---------+----------+-------------+-------------------+-------------+---------------+------------------+
|1          |Caitlyn  |103          |115959.78|2002-06-10|103          |Sales and Marketing|Chicago      |High           |22.775342465753425|
|2          |Rachel   |104          |100820.16|2009-07-01|104          |Data Engineering   |Zapopan      |High           |15.712328767123287|
|3          |Carrie   |105          |114421.44|1998-12-10|105          |Data Science       |Seattle      |High           |26.276712328767122|
|4          |Renee    |104          |54688.13 |1995-03-17|104          |Data Engineering   |Zapopan      |Low            |30.013698630136986|
|5    

In [48]:
employee_count = employees_older_df.count()

print(employee_count)

2261
